## Data collection and imputation

In [56]:
%load_ext autoreload
%autoreload 2

In [57]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from fantasy_football import Fantasy

# Import fantasy class which will be used for building datasets
football = Fantasy()

First, we pull 2022 offense data and clean it

In [58]:
df = pd.read_excel('2022-offense.xlsx', sheet_name='Offense_Prior_Actuals', header=3)

# Drop columns that we don't need
drop_columns = ['GS', 'Bye', 'Notes', 'Rank', 'Y! Roto', 'Δ', 'Y! Old', 'Std', 'Δ.1', 'Std Old', 'PPR', 'Δ.2', 'PPR Old', '% Own', 'Fan Pts', 'PPG']
df = df.drop(drop_columns, axis=1)

# Change names of duplicate columns
updated_columns = {'Yds': 'Passing_Yds', 'Yds.1': 'Rushing_Yds', 'Yds.2': 'Receiving_Yds', 'Yds.3': 'Return_Yds', 
                   'TD': 'Passing_Td', 'TD.1': 'Rushing_Td', 'TD.2': 'Receiving_Td', 'TD.3': 'Return_Td', 
                   '1st': 'Passing_1st', '1st.1': 'Rushing_1st', '1st.2': 'Receiving_1st'}
df = df.rename(columns=updated_columns)


Split data by positional group

In [65]:
QB_data, RB_data, WR_data, TE_data = football.makeRegularData(df)


Next, we pull more advanced data for rushing and passing

In [59]:
# Passing advanced
passing_advanced = football.getAdvancedStats("passing", "2021")

# Rushing advanced
rushing_advanced = football.getAdvancedStats("rushing", "2021")

Now we are ready to merge the datasets to incorporate the more advance statistics

In [63]:
# Merge Datasets
qb_merged = football.addAdvanced(QB_data, passing_advanced)
rb_merged = football.addAdvanced(RB_data, rushing_advanced)

,Player_x,Pos_x,Team,GP,Att_x,Rushing_Yds,Rushing_Td,Rushing_1st,Tgt,Rec,...,Att_y,Yds,TD,1D,YBC,YBC/Att,YAC,YAC/Att,BrkTkl,Att/Br
0,J. Taylor,RB,Ind,17,332,1811,18,107,51,40,...,332,1811,18,107,870,2.6,941,2.8,25,13.3
1,J. Taylor,RB,Ind,17,332,1811,18,107,51,40,...,23,89,1,5,53,2.3,36,1.6,2,11.5
2,J. Taylor,RB,Ind,17,332,1811,18,107,51,40,...,19,37,2,3,25,1.3,12,0.6,0,
3,J. Taylor,RB,Ind,17,332,1811,18,107,51,40,...,19,151,3,6,86,4.5,65,3.4,4,4.8
4,P. Taylor,RB,GB,9,23,89,1,5,2,2,...,332,1811,18,107,870,2.6,941,2.8,25,13.3


### CONTRACTS AND OTHER DATA (not yet completed)

We are not using the player contracts yet but here is where we scrape that data

In [62]:
table_contracts = pd.read_html('https://www.spotrac.com/nfl/contracts/sort-value/offense/all-time/free-agent-2021/limit-100/')

player_contracts = table_contracts[0]
player_contracts

,Rank,Player,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed
0,1,Jay Cutler Quarterback | 2014-2020 (FA: 2021)...,30,7,"$126,700,000","$18,100,000",NaN,"$38,000,000","$54,000,000"
1,2,Cam Newton Quarterback | 2015-2020 (FA: 2021),26,5,"$103,800,000","$20,760,000","$22,500,000","$41,000,000","$60,000,000"
2,3,Andy Dalton Quarterback | 2014-2020 (FA: 2021...,26,6,"$96,000,000","$16,000,000","$12,000,000","$17,000,000","$17,000,000"
3,4,Kirk Cousins Quarterback | 2018-2020 (FA: 2021),29,3,"$84,000,000","$28,000,000","$3,000,000","$84,000,000","$84,000,000"
4,5,Ryan Tannehill Quarterback | 2015-2020 (FA: 2...,26,4,"$77,000,000","$19,250,000","$11,500,000","$21,500,000","$45,000,000"
...,...,...,...,...,...,...,...,...,...
95,96,Paxton Lynch Quarterback | 2016-2020 (FA: 202...,22,4,"$9,476,296","$2,369,074","$5,091,852","$7,884,074","$7,884,074"
96,97,Lee Smith Tight End | 2018-2020 (FA: 2021) O...,30,3,"$9,000,000","$3,000,000",NaN,"$3,000,000","$3,000,000"
97,97,Sammy Watkins Wide Receiver | 2020-2020 (FA: ...,26,1,"$9,000,000","$9,000,000","$1,000,000","$8,000,000","$8,000,000"
98,97,David Andrews Center | 2017-2020 (FA: 2021),24,3,"$9,000,000","$3,000,000","$1,600,000","$1,600,000","$1,600,000"
